*Warning : This is an experimental kernel. There are wrong information.*

*Note: https://www.kaggle.com/danilodiogo/the-astronomical-complete-eda-plasticc-dataset convert to python code*

In [ ]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import gc

- `[training/test]_set_metadata`: Information about objects in the that doesn't change over time, like the coordinates of the object.
- `[training/test]_set`: Time series of observations of the objects. Maps to the metadata via object_id.

In [ ]:
train = pd.read_csv("../input/training_set.csv")
train_meta = pd.read_csv("../input/training_set_metadata.csv")

### Metadata File Column Descriptions

In [ ]:
train_meta.head()

- `object_id`: unique object identifier. Int32
- `ra`: right ascension, sky coordinate: co-longitude in degrees. Float32
- `decl`: declination, sky coordinate: co-latitude in degrees. Float32
- `gal_l`: galactic longitude in degrees. Float32
- `gal_b`: galactic latitude in degrees. Float32
- `ddf`: A flag to identify the object as coming from the DDF survey area (with value DDF = 1 for the DDF, DDF = 0 for the WFD survey). Note that while the - DDF fields are contained within the full WFD survey area, the DDF fluxes have significantly smaller uncertainties. Boolean
- `hostgal_specz`: the spectroscopic redshift of the source. This is an extremely accurate measure of redshift, available for the training set and a small fraction of the test set. Float32 (적색편이)
- `hostgal_photoz`: The photometric redshift of the host galaxy of the astronomical source. While this is meant to be a proxy for hostgal_specz, there can be large differences between the two and should be regarded as a far less accurate version of hostgal_specz. Float32
- `hostgal_photoz_err`: The uncertainty on the hostgal_photoz based on LSST survey projections. Float32
- `distmod`: The distance to the source calculated from hostgal_photoz and using general relativity. Float32
- `mwebv`: MW E(B-V). this ‘extinction’ of light is a property of the Milky Way (MW) dust along the line of sight to the astronomical source, and is thus a function of the sky coordinates of the source ra, decl. This is used to determine a passband dependent dimming and redenning of light from astronomical sources as described in subsection 2.1, and based on the Schlafly et al. (2011) and Schlegel et al. (1998) dust models. Float32
- `target`: The class of the astronomical source. This is provided in the training data. Correctly determining the target (correctly assigning classification probabilities to the objects) is the ‘goal’ of the classification challenge for the test data. Note that there is one class in the test set that does not occur in the training set: class_99 serves as an "other" class for objects that don't belong in any of the 14 classes in the training set. Int8

![](https://upload.wikimedia.org/wikipedia/commons/9/98/Ra_and_dec_on_celestial_sphere.png)

**Right ascension (abbreviated RA; symbol α)** is the angular distance measured only eastward along the celestial equator from the Sun at the March equinox to the (hour circle of the) point above the earth in question. When paired with **declination**, these astronomical coordinates specify the direction of a point on the celestial sphere in the equatorial coordinate system.

![](http://upload.wikimedia.org/wikipedia/commons/2/2a/Galactic_coordinates.JPG)

**Galactic longitude (l)** is measured with primary direction from the Sun to the center of the galaxy in the galactic plane, while the **galactic latitude (b)** measures the angle of the object above the galactic plane.

**redshift** : when light or other electromagnetic radiation from an object is increased in wavelength, or shifted to the red end of the spectrum. In general, whether or not the radiation is within the visible spectrum, "redder" means an increase in wavelength – equivalent to a lower frequency and a lower photon energy, in accordance with, respectively, the wave and quantum theories of light.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/High-redshift_galaxy_candidates_in_the_Hubble_Ultra_Deep_Field_2012.jpg/800px-High-redshift_galaxy_candidates_in_the_Hubble_Ultra_Deep_Field_2012.jpg)

Resource : https://en.wikipedia.org/wiki/Redshift

**spectroscopic redshift** vs **photometric redshift** : 

Resource : https://physics.stackexchange.com/questions/169734/photometric-surveys-vs-spectroscopic-surveys

**LSST** : 
https://www.lsst.org/scientists/survey-design


In [ ]:
print("unique values of object_id:",len(train_meta['object_id'].unique()),"\nshape of train_meta:",train_meta.shape[0])

In [ ]:
#missing data
total = train_meta.isnull().sum().sort_values(ascending=False)
percent = (train_meta.isnull().sum()/train_meta.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

#histogram
#missing_data = missing_data.head(20)
percent_data = percent.head(20)
percent_data.plot(kind="bar", figsize = (10,8), fontsize = 15)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Percent of Missing Value (%)", fontsize = 20)
#plt.title("Total Missing Value (%)", fontsize = 20)

- `distmod`: The distance to the source calculated from hostgal_photoz and using general relativity. Float32
- `hostgal_photoz`: The photometric redshift of the host galaxy of the astronomical source. While this is meant to be a proxy for hostgal_specz, there can be large differences between the two and should be regarded as a far less accurate version of hostgal_specz. Float32

In [ ]:
train_meta[train_meta['distmod'].isnull()].head()

- hostgal_specz, hostgal_photoz, hostgal_photoz_err have no value. it is mean that the object has no redshift value. so don't calculate distmod.

Comment in the https://www.kaggle.com/danilodiogo/the-astronomical-complete-eda-plasticc-dataset , Danilo da Silva makes the variable `Milky Way` because the dataset introduction file says "the given redshift for objects in our own Milky Way galaxy is given as zero".

In [ ]:
pd.isna(train_meta['distmod'].iloc[0])

In [ ]:
train_meta['Milky.Way'] = train_meta["distmod"].apply(lambda x: 1 if pd.isnull(x) == True else 0)

In [ ]:
train_meta.head()

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
train_meta['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.show()

Target value is uneven. it seems class imbalance. I wonder if I should use sampling here.

In [ ]:
g = sns.FacetGrid(train_meta, hue = 'target',height=8)
g.map(plt.scatter, 'gal_l','gal_b')

Whale...?

In [ ]:
g = sns.FacetGrid(train_meta, col = 'target',col_wrap=4)
g.map(plt.scatter, 'gal_l','gal_b')

In [ ]:
g = sns.FacetGrid(train_meta, hue = 'target',height=8)
g.map(plt.scatter, 'ra','decl')

In [ ]:
g = sns.FacetGrid(train_meta, col = 'target',col_wrap=4)
g.map(plt.scatter, 'ra','decl')

![](https://steemitimages.com/DQmQR7j1yGHYhP1sheeGprjyYEiYG7dSfft1z1jJbHpsy6v/9.PNG)

In [ ]:
np.sin(349.046051 * np.pi / 180)

In [ ]:
train_meta['skycoordinate_x'] = train_meta["distmod"] * train_meta["decl"].apply(lambda x: np.sin(x * np.pi/180)) * train_meta["ra"].apply(lambda x: np.cos(x * np.pi/180))
train_meta['skycoordinate_y'] = train_meta["distmod"] * train_meta["decl"].apply(lambda x: np.sin(x * np.pi/180)) * train_meta["ra"].apply(lambda x: np.sin(x * np.pi/180))
train_meta['skycoordinate_z'] = train_meta["distmod"] * train_meta["decl"].apply(lambda x: np.cos(x * np.pi/180))

In [ ]:
#https://matplotlib.org/examples/mplot3d/scatter3d_demo.html
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,8))
ax = Axes3D(fig)

#ax.scatter(swasp_mag1,swasp_per1,swasp_age1,edgecolor='none',c='r',marker='o',s=35,label='SWASP')
#ax.scatter(hyd_mag1,hyd_per1,hyd_age1,edgecolor='none',c='y',marker='o',s=35,label='Hyades')
#ax.scatter(pld_mag1,pld_per1,pld_age1,edgecolor='none',c='b',marker='o',s=35,label='Pleiades')

ax.scatter(train_meta[train_meta['target']==90]['skycoordinate_x'], train_meta[train_meta['target']==90]['skycoordinate_y'], train_meta[train_meta['target']==90]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==42]['skycoordinate_x'], train_meta[train_meta['target']==42]['skycoordinate_y'], train_meta[train_meta['target']==42]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==65]['skycoordinate_x'], train_meta[train_meta['target']==65]['skycoordinate_y'], train_meta[train_meta['target']==65]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==16]['skycoordinate_x'], train_meta[train_meta['target']==16]['skycoordinate_y'], train_meta[train_meta['target']==16]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==15]['skycoordinate_x'], train_meta[train_meta['target']==15]['skycoordinate_y'], train_meta[train_meta['target']==15]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==62]['skycoordinate_x'], train_meta[train_meta['target']==62]['skycoordinate_y'], train_meta[train_meta['target']==62]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==88]['skycoordinate_x'], train_meta[train_meta['target']==88]['skycoordinate_y'], train_meta[train_meta['target']==88]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==92]['skycoordinate_x'], train_meta[train_meta['target']==92]['skycoordinate_y'], train_meta[train_meta['target']==92]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==67]['skycoordinate_x'], train_meta[train_meta['target']==67]['skycoordinate_y'], train_meta[train_meta['target']==67]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==52]['skycoordinate_x'], train_meta[train_meta['target']==52]['skycoordinate_y'], train_meta[train_meta['target']==52]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==95]['skycoordinate_x'], train_meta[train_meta['target']==95]['skycoordinate_y'], train_meta[train_meta['target']==95]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==6]['skycoordinate_x'], train_meta[train_meta['target']==6]['skycoordinate_y'], train_meta[train_meta['target']==6]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==64]['skycoordinate_x'], train_meta[train_meta['target']==64]['skycoordinate_y'], train_meta[train_meta['target']==64]['skycoordinate_z'])
ax.scatter(train_meta[train_meta['target']==53]['skycoordinate_x'], train_meta[train_meta['target']==53]['skycoordinate_y'], train_meta[train_meta['target']==53]['skycoordinate_z'])

plt.title("sky coordinate")
plt.show()

In [ ]:
train_meta['galatic_x'] = train_meta["distmod"] * train_meta["gal_b"].apply(lambda x: np.sin(x * np.pi/180)) * train_meta["gal_l"].apply(lambda x: np.cos(x * np.pi/180))
train_meta['galatic_y'] = train_meta["distmod"] * train_meta["gal_b"].apply(lambda x: np.sin(x * np.pi/180)) * train_meta["gal_l"].apply(lambda x: np.sin(x * np.pi/180))
train_meta['galatic_z'] = train_meta["distmod"] * train_meta["gal_b"].apply(lambda x: np.cos(x * np.pi/180))

In [ ]:
#https://matplotlib.org/examples/mplot3d/scatter3d_demo.html

fig = plt.figure(figsize=(10,8))
ax = Axes3D(fig)

#ax.scatter(swasp_mag1,swasp_per1,swasp_age1,edgecolor='none',c='r',marker='o',s=35,label='SWASP')
#ax.scatter(hyd_mag1,hyd_per1,hyd_age1,edgecolor='none',c='y',marker='o',s=35,label='Hyades')
#ax.scatter(pld_mag1,pld_per1,pld_age1,edgecolor='none',c='b',marker='o',s=35,label='Pleiades')

ax.scatter(train_meta[train_meta['target']==90]['galatic_x'], train_meta[train_meta['target']==90]['galatic_y'], train_meta[train_meta['target']==90]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==42]['galatic_x'], train_meta[train_meta['target']==42]['galatic_y'], train_meta[train_meta['target']==42]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==65]['galatic_x'], train_meta[train_meta['target']==65]['galatic_y'], train_meta[train_meta['target']==65]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==16]['galatic_x'], train_meta[train_meta['target']==16]['galatic_y'], train_meta[train_meta['target']==16]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==15]['galatic_x'], train_meta[train_meta['target']==15]['galatic_y'], train_meta[train_meta['target']==15]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==62]['galatic_x'], train_meta[train_meta['target']==62]['galatic_y'], train_meta[train_meta['target']==62]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==88]['galatic_x'], train_meta[train_meta['target']==88]['galatic_y'], train_meta[train_meta['target']==88]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==92]['galatic_x'], train_meta[train_meta['target']==92]['galatic_y'], train_meta[train_meta['target']==92]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==67]['galatic_x'], train_meta[train_meta['target']==67]['galatic_y'], train_meta[train_meta['target']==67]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==52]['galatic_x'], train_meta[train_meta['target']==52]['galatic_y'], train_meta[train_meta['target']==52]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==95]['galatic_x'], train_meta[train_meta['target']==95]['galatic_y'], train_meta[train_meta['target']==95]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==6]['galatic_x'], train_meta[train_meta['target']==6]['galatic_y'], train_meta[train_meta['target']==6]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==64]['galatic_x'], train_meta[train_meta['target']==64]['galatic_y'], train_meta[train_meta['target']==64]['galatic_z'])
ax.scatter(train_meta[train_meta['target']==53]['galatic_x'], train_meta[train_meta['target']==53]['galatic_y'], train_meta[train_meta['target']==53]['galatic_z'])

plt.title("galatic coordinate")
plt.show()

In [ ]:
train_meta['ddf'].plot()

hmm...?

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
train_meta['ddf'].value_counts().sort_values(ascending=False).plot.bar()
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
train_meta[train_meta['ddf']==1]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title("ddf is 1 of target")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
train_meta[train_meta['ddf']==0]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title("ddf is 0 of target")
plt.show()

In [ ]:
g = sns.FacetGrid(train_meta, hue = 'ddf',height=8)
g.map(plt.scatter, 'gal_l','gal_b')

In [ ]:
g = sns.FacetGrid(train_meta, hue = 'ddf',height=8)
g.map(plt.scatter, 'ra','decl')

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
fig = sns.boxplot(x='target', y="hostgal_specz", data=train_meta)

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
fig = sns.boxplot(x='target', y="hostgal_photoz", data=train_meta)

In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(14,12),sharey=True)
sns.boxplot(x='target', y="hostgal_specz", data=train_meta,ax=axis1)
axis1.set_title('Spectroscopic Redshift')
sns.boxplot(x='target', y="hostgal_photoz", data=train_meta,ax=axis2)
axis2.set_title('Photometric Redshift')

Photometric seems to more outliers

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
fig = sns.boxplot(x='target', y="distmod", data=train_meta)

target 6, 16, 53, 65, 92 has no distmod 

In [ ]:
train_meta[train_meta['distmod'].isnull()]['target'].unique()

In [ ]:
g = sns.FacetGrid(train_meta, hue = 'Milky.Way',height=8)
g.map(plt.scatter, 'gal_l','gal_b')

In [ ]:
g = sns.FacetGrid(train_meta, hue = 'Milky.Way',height=8)
g.map(plt.scatter, 'ra','decl')

In [ ]:
g = sns.FacetGrid(train_meta, col = 'Milky.Way',col_wrap=2)
g.map(plt.scatter, 'gal_l','gal_b')

In [ ]:
g = sns.FacetGrid(train_meta, col = 'Milky.Way',col_wrap=2)
g.map(plt.scatter, 'ra','decl')

In [ ]:
print("Milky.Way is 0 of Target:",train_meta[train_meta['Milky.Way']==0]['target'].unique(),"\nMilky.Way is 1 of Target:",train_meta[train_meta['Milky.Way']==1]['target'].unique())

### Train File Column Descriptions

In [ ]:
train.head()

- object id: Same key as in the metadata table abov.
- mjd: the time in Modified Julian Date (MJD) of the observation.
- passband: The specific LSST passband integer, such that u, g, r, i, z, y = 0, 1, 2, 3, 4, 5 in which it was viewed.
- flux: the measured flux (brightness) in the passband of observation as listed in the passband column. The flux is corrected for MWEBV.
- flux err: the uncertainty on the measurement of the flux listed above.
detected: If detected= 1, the object’s brightness is significantly different at the 3σ level relative to the reference template.

In [ ]:
train.describe()

In [ ]:
train['object_id'] = train['object_id'].astype(np.object)
train_meta['object_id'] = train_meta['object_id'].astype(np.object)

In [ ]:
train['object_id'].head()

In [ ]:
train_all = train_meta.merge(train, how='left', on='object_id')

In [ ]:
train_all.head()

In [ ]:
del train, train_meta
gc.collect()

In [ ]:
train_all['mjd'].head()

In [ ]:
train_all['date'] = pd.to_datetime((train_all['mjd']-40587)*86400,unit='s')

In [ ]:
train_all.head()

In [ ]:
train_all['target'].unique()

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
import random

#https://www.kaggle.com/ashishpatel26/bird-eye-view-of-two-sigma-xgb
def generate_color():
    color = '#{:02x}{:02x}{:02x}'.format(*map(lambda x: random.randint(0, 255), range(3)))
    return color
for i in [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]:
    
    data = []
    for j in [0,1,2,3,4,5]:
        j = go.Scatter(
                x = train_all[(train_all['target']==i)]['date'].dt.strftime(date_format='%Y-%m-%d').values,
                y = train_all[(train_all['target']==i) & (train_all['passband']==j)]['flux'].values,
                line = dict(color = generate_color()),opacity = 0.8
            )

        data.append(j)
                    
    layout = dict(title = "Target {}".format(i),
                          xaxis = dict(title = 'date'),
                          yaxis = dict(title = 'flux'),
                          )
    py.iplot(dict(data=data, layout=layout), filename='basic-line')

If you click the trace on the upper right, you can see more better.

In [ ]:
fig, (axes) = plt.subplots(2,3,figsize=(18,12),sharey=True)
sns.boxplot(x='target', y="flux_err", data=train_all[train_all['passband']==0],ax=axes[0,0])
axes[0,0].set_title('0')
sns.boxplot(x='target', y="flux_err", data=train_all[train_all['passband']==1],ax=axes[0,1])
axes[0,1].set_title('1')
sns.boxplot(x='target', y="flux_err", data=train_all[train_all['passband']==2],ax=axes[0,2])
axes[0,2].set_title('2')
sns.boxplot(x='target', y="flux_err", data=train_all[train_all['passband']==3],ax=axes[1,0])
axes[1,0].set_title('3')
sns.boxplot(x='target', y="flux_err", data=train_all[train_all['passband']==4],ax=axes[1,1])
axes[1,1].set_title('4')
sns.boxplot(x='target', y="flux_err", data=train_all[train_all['passband']==5],ax=axes[1,2])
axes[1,2].set_title('5')

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
train_all['detected'].value_counts().sort_values(ascending=False).plot.bar()
plt.show()

In [ ]:
fig, (axes) = plt.subplots(2,2,figsize=(18,12))
ax1 = plt.subplot(2,2,1)
train_all[(train_all['detected']==0) & (train_all['ddf']==0)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('detected : 0 & ddf : 0')
ax1 = plt.subplot(2,2,2)
train_all[(train_all['detected']==0) & (train_all['ddf']==1)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('detected : 0 & ddf : 1')
ax1 = plt.subplot(2,2,3)
train_all[(train_all['detected']==1) & (train_all['ddf']==0)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('detected : 1 & ddf : 0')
ax1 = plt.subplot(2,2,4)
train_all[(train_all['detected']==1) & (train_all['ddf']==1)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('detected : 1 & ddf : 1')

In [ ]:
fig, (axes) = plt.subplots(2,2,figsize=(18,12))
ax1 = plt.subplot(2,2,1)
train_all[(train_all['detected']==0) & (train_all['Milky.Way']==0)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('detected : 0 & Milky.Way : 0')
ax1 = plt.subplot(2,2,2)
train_all[(train_all['detected']==0) & (train_all['Milky.Way']==1)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('detected : 0 & Milky.Way : 1')
ax1 = plt.subplot(2,2,3)
train_all[(train_all['detected']==1) & (train_all['Milky.Way']==0)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('detected : 1 & Milky.Way : 0')
ax1 = plt.subplot(2,2,4)
train_all[(train_all['detected']==1) & (train_all['Milky.Way']==1)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('detected : 1 & Milky.Way : 1')

In [ ]:
fig, (axes) = plt.subplots(2,2,figsize=(18,12))
ax1 = plt.subplot(2,2,1)
train_all[(train_all['ddf']==0) & (train_all['Milky.Way']==0)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('ddf : 0 & Milky.Way : 0')
ax1 = plt.subplot(2,2,2)
train_all[(train_all['ddf']==0) & (train_all['Milky.Way']==1)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('ddf : 0 & Milky.Way : 1')
ax1 = plt.subplot(2,2,3)
train_all[(train_all['ddf']==1) & (train_all['Milky.Way']==0)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('ddf : 1 & Milky.Way : 0')
ax1 = plt.subplot(2,2,4)
train_all[(train_all['ddf']==1) & (train_all['Milky.Way']==1)]['target'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('ddf : 1 & Milky.Way : 1')